In [42]:
import os
import pickle
from nltk.translate import bleu_score

# Load utils

In [39]:
from utils import load_vocab, decode_caption, load_caption, rrv_votes, load_annotations, print_image

vocab = load_vocab(dict_file = "../../outputs/vocab/5000/coco2014_vocab.json")
image_id_to_index, index_to_image_id, annotations_dict = load_annotations(annotations_dir="../../annotations/", 
                                                                          annotations_file='captions_val2014.json',
                                                                          map_file = "../../outputs/val_image_id_to_idx.csv")
print("Processed {} images".format(len(image_id_to_index)))
print("Processed {} images".format(len(annotations_dict.keys())))

annotations_list = [annotations_dict[image_id] for image_id in sorted(annotations_dict)]

word_to_idx
idx_to_word
Loaded dictionary...
Dictionary size: 5004
Error proccessing image_id: image_index
Skipping file person_keypoints_train2014.json
Skipping file instances_train2014.json
Skipping file instances_val2014.json
Skipping file person_keypoints_val2014.json
Processed 40504 images
Processed 40504 images


In [54]:
def load_best_beam_captions(beam_size):
    file_name = '../../outputs/voted_captions/{}/best_beam.pickle'.format(beam_size)
    with open(file_name, 'rb') as file:
        best_beams = pickle.load(file)
    return [decode_caption(best_beams[im_id], vocab) for im_id in sorted(best_beams)]

# Load all beams and extract best for each caption

In [74]:
def load_diverse_beam(diverse_beam_path):
    with open(diverse_beam_path, 'rb') as file:
        beams = pickle.load(file)
    return beams

def load_best_diverse_beams(beam_size):
    best_beams = {}
    beam_captions_dir = "../../outputs/diverse_beam/diverse_{}/".format(beam_size)
    for beam_path in os.listdir(beam_captions_dir):
        image_id = int(beam_path.split('.')[0])
        beam = load_diverse_beam(os.path.join(beam_captions_dir, beam_path))
        assert image_id == beam['image_id']
        best_captions = decode_caption(beam['captions'][0], vocab)
        best_beams[image_id] = best_captions
    return best_beams

## Compute Bleu score

In [82]:
def bleu(best_beams):
    filtered_annotations = []
    captions = []
    for k in best_beams.keys():
        filtered_annotations.append(annotations_list[k])
        captions.append(best_beams[k])
    return bleu_score.corpus_bleu(filtered_annotations, captions)

In [83]:
diverse_best_beams = load_best_diverse_beams(10)
diverse_bleu = bleu(diverse_best_beams)

In [84]:
best_beam = load_best_beam_captions(10)
filtered_best_beam = {k: best_beam[k] for k in diverse_best_beams.keys()}
beam_bleu = bleu(filtered_best_beam)

In [85]:
print("Diverse bleu", diverse_bleu)
print("Beam bleu", beam_bleu)

Diverse bleu 0.26893349633588554
Beam bleu 0.27322415240348363


# Load best diverse beam captions

In [1]:
# def load_diverse_beam_captions(beam_size):
# #     beam_captions = {}
# #     beam_captions_dir = "../../outputs/beam_captions_{}/".format(beam_size)
# #     for image_id in sorted(annotations_dict):
# #         caption_object = load_caption(image_id, image_dir=beam_captions_dir)
# #         best_beam_caption = caption_object['captions'][0]['sentence']
# #         beam_captions[image_id] = best_beam_caption
# #     return beam_captions
#     file_name = '../../outputs/voted_captions/{}/best_beam.pickle'.format(beam_size)
# #     with open(file_name, 'wb') as file:
# #         pickle.dump(beam_captions, file, pickle.HIGHEST_PROTOCOL)
#     with open(file_name, 'rb') as file:
#         best_beams = pickle.load(file)
#     return [decode_caption(best_beams[im_id], vocab) for im_id in sorted(best_beams)]